<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/06_combined_model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-27 02:01:53--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12110 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.83K  --.-KB/s    in 0s      

2022-06-27 02:01:53 (70.1 MB/s) - ‘utils.py’ saved [12110/12110]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 58.6 MB/s 


In [4]:
BATCH_SIZE = 16
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
rgb_input = keras.Input(shape=IMG_SIZE+(3,))
rgb_hub_module = hub.KerasLayer(HUB_URL,  trainable=True)
hsv_hub_module = hub.KerasLayer(HUB_URL,  trainable=True)
xyz_hub_module = hub.KerasLayer(HUB_URL,  trainable=True)

rgb_features = rgb_hub_module(rgb_input)

xyz_input = tfio.experimental.color.rgb_to_xyz(rgb_input)
xyz_features = xyz_hub_module(xyz_input)

hsv_input = tf.image.rgb_to_hsv(rgb_input)
hsv_features = hsv_hub_module(hsv_input)

features = keras.layers.Concatenate()([rgb_features, xyz_features, hsv_features])

output = keras.layers.Dense(N_CLASSES, activation='softmax')(features)

model = keras.models.Model(inputs=[rgb_input], outputs=[output])

In [10]:
model._name = 'combined-model-v2'

In [11]:
metrics = [
            keras.metrics.AUC(name='auc', curve='PR', num_thresholds=100),
            'accuracy'
          ]

loss_fn = keras.losses.CategoricalCrossentropy()
lr_schedule =tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 500, .9)

model.compile(loss=loss_fn,
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=metrics)

In [13]:
early_stopping = keras.callbacks.EarlyStopping(
                                              monitor='val_accuracy', 
                                              verbose=1,
                                              patience=8,
                                              mode='max')

check_point_path = os.path.join('./logs/checkpoints', model._name)
check_point = keras.callbacks.ModelCheckpoint(
                                            filepath=check_point_path,
                                            monitor='val_accuracy',
                                            save_best_only=True,
                                            mode='max')
        
callbacks = [early_stopping, check_point]

In [14]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=15,
                    validation_data=val_ds,
                    class_weight=class_weight)

Epoch 1/15
1037/1037 [==============================] - ETA: 0s - loss: 19.1490 - auc: 0.7048 - accuracy: 0.6300

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


1037/1037 [==============================] - 3393s 3s/step - loss: 19.1490 - auc: 0.7048 - accuracy: 0.6300 - val_loss: 0.9229 - val_auc: 0.7616 - val_accuracy: 0.6687
Epoch 2/15
1037/1037 [==============================] - ETA: 0s - loss: 11.7602 - auc: 0.8454 - accuracy: 0.7602

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


...

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


1037/1037 [==============================] - 741s 714ms/step - loss: 0.5622 - auc: 0.9975 - accuracy: 0.9855 - val_loss: 0.9217 - val_auc: 0.8661 - val_accuracy: 0.8329
Epoch 11/15
1037/1037 [==============================] - 688s 662ms/step - loss: 0.4347 - auc: 0.9981 - accuracy: 0.9888 - val_loss: 1.0412 - val_auc: 0.8507 - val_accuracy: 0.8286
Epoch 12/15
1037/1037 [==============================] - 686s 661ms/step - loss: 0.3499 - auc: 0.9983 - accuracy: 0.9913 - val_loss: 1.1656 - val_auc: 0.8365 - val_accuracy: 0.8226
Epoch 13/15
1037/1037 [==============================] - 686s 661ms/step - loss: 0.2164 - auc: 0.9987 - accuracy: 0.9949 - val_loss: 1.2108 - val_auc: 0.8295 - val_accuracy: 0.8266
Epoch 14/15
1037/1037 [==============================] - 688s 662ms/step - loss: 0.2011 - auc: 0.9994 - accuracy: 0.9955 - val_loss: 1.1531 - val_auc: 0.8394 - val_accuracy: 0.8323
Epoch 15/15
1037/1037 [==============================] - ETA: 0s - loss: 0.1422 - auc: 0.9993 - accuracy: 0

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


1037/1037 [==============================] - 740s 713ms/step - loss: 0.1422 - auc: 0.9993 - accuracy: 0.9964 - val_loss: 1.1354 - val_auc: 0.8389 - val_accuracy: 0.8355


In [15]:
!cp  -r /content/logs/checkpoints/combined-model-v2 /content/drive/MyDrive/checkpoints/combined-v2